# CREATING THE LIST OF PRODUCTS FOR THE ANALYSIS:

After rearranging the data in a more convenient manner and doing some introductory analysis of the data, we now want to get down to work with our data.

A list has been given to us of the 10 products that our clients finded as more relevant to their business.

What we want now is to check wether the names on the list correspond to certain uniques ids, or, as seen in the previous scripts, some conflict of unicity will arise between the id of our products and their descriptions.

So, we are going to check our dataframe and select from it the ids and descriptions of our products that match the indications given in our clients list.

We will do this in two phases: in the first one we will create a dataframe, with only some of the rows of our csv.

In the second part, we will use chunks to load, chunk by chunk, all the dataframe, and extract the samei nformation.

Finally, with the lists (in reality, two dictionaries) of the ids and descriptions that match every product given to us, we will decide which are the more appropriate.

Perhaps some guidance from our client would be needed at this stage.

## 1. LOADING ONLY PART OF THE DATAFRAME:

In [1]:
# Importing packages:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
# Defining the search path of the file, the name and the separator:

file_path = "../../data/01_raw/"
file_name = "1_b2-transactions.csv"

sep=";"

In [3]:
# We create the list of selected products, and will try with it to obtain the values of the ids of the indicated products:

list_of_products=['croissant',
                  'croissant petit',
                  'tarta mousse 3 chocolates',
                  'tarta de manzana 2º',
                  'palmera', 
                  'tarta opera',
                  'postre fresas y mascarpone',
                  'milhojas frambuesa 2º',
                  'tortel',
                  'baguette']

In [4]:
# We import the dataframe:

df=pd.read_csv(file_path+file_name, nrows=1000000, sep=sep)

In [5]:
# According to what we saw in the previous notebook, we have to do some cleaning:

df.dropna(how='any', inplace=True)
# df=df.drop('Unnamed: 0', axis=1)

In [6]:
# Most of the descriptions are in uppercase, we first reduce everything to lowercase:

df['description_lower']=df['description'].str.lower()

In [7]:
# Now we construct the dictionaries of the selected ids and descriptions that each product of the list finds:

# What you are seeing here is the final version both of the list of the selected products, and the way they are looked for in the
# dataframe. We started looking within the string of the description, with a ".contains", but it was noticed that startswith 
# was more suited for the job. Also, the values in the list of the products were changed in order to capture more names
# each time:

rel_prod_list_ids=dict()
rel_prod_list_descrip=dict()

for product in list_of_products:
    rel_prod_list_ids[product]=df[df['description_lower'].str.startswith(product)]['product_id'].unique()
    rel_prod_list_descrip[product]=df[df['description_lower'].str.startswith(product)]['description_lower'].unique()

In [8]:
# For minor, additional checks:

df[df['product_id']==107].groupby('description').first()

,product_id,order_date,section,store,units_ordered,description_lower
description,,,,,,
CROISSANT ALMENDRA LARGO,107.0,2019-09-03,0,BmUP,"0,00",croissant almendra largo
CROISSANT VACIOS,107.0,2009-06-16,0,BmUP,"3,00",croissant vacios


In [9]:
# This is the dictionary with the word and its associated codes:

rel_prod_list_ids

{'croissant': array([ 102.,  103.,  105.,  107.,  101.,  132., 5001.,  100.,  214.,
         189.,  198.,  197., 9999.,  513.,  512.,  112.]),
 'croissant petit': array([103., 102.]),
 'tarta mousse 3 chocolates': array([9999.,  453.]),
 'tarta de manzana 2º': array([462.]),
 'palmera': array([ 140.,  182.,  190., 9999.,  141.]),
 'tarta opera': array([ 9999.,   414.,   426.,   427.,   403., 14998.,   402.]),
 'postre fresas y mascarpone': array([4511., 9999.]),
 'milhojas frambuesa 2º': array([459.]),
 'tortel': array([ 112., 3352., 9999., 3375.]),
 'baguette': array([ 115., 8739., 9999.])}

In [10]:
# The same but with the descriptions: Just for checks:

rel_prod_list_descrip[list_of_products[5]]

array(['tarta opera del 2 escrito sobre la trta felicidades rafael ',
       'tarta opera del 2º con cartel "felicidades"',
       'tarta opera 5º con cartel "felicidades lili" y adornada con frutas naturales',
       'tarta opera 2º', 'tarta opera del 5º',
       'tarta opera del 4º con cartel " felicidades dolly "',
       'tarta opera del 2º con cartel " felicidades raul "',
       'tarta opera del 4º con cartel que ponga "felicidades gaës" (ojo que la letra e lleva dieresis)',
       'tarta opera del 2º escrito encima " happy birthay nano  aba y papa "',
       'tarta opera 2º escrito en un cartel felicidades 18',
       'tarta opera 3º',
       'tarta opera 32 rac. escrito sobre la tarta " felicidades jose. feliz 60 cumpleaños"ccccccccccc',
       'tarta opera del 4  con cartel escrito- hugo y mar, muchas felicidades de vuestra familia-',
       'tarta opera 5º', 'tarta opera 2º felicidades alejandra',
       'tarta opera del 6º escrito feliz cumpleaños',
       'tarta opera 10 ra

In [11]:
# For some additional checks:

df[df['product_id']==450]['description'].unique()

array(['POSTRE MOUSSE TRES CHOCOLATES',
       'POSTRE VIRUTA CHOCOLATE  RECTANGULAR',
       'POSTRES  MILHOJAS  NATA CREMA', ...,
       'POSTRE MANZANA  CARAMELO -- HOTELES-',
       'POSTRE MOUSSE CHOCOLATE  VASITO',
       'POSTRE RECTANGULAR  VIRUTA CHOCOLATE'], dtype=object)

## 2. LOADING ALL THE DATAFRAME IN CHUNKS AND GETTING ALL THE RESULTS:

In [12]:
# We import the dataframe:

reader=pd.read_csv(file_path+file_name, sep=sep, chunksize=2000000)

# Two empty lists are created to store, for each product, the list that results from looking for it in the chunk.

NL1=list([None]*len(list_of_products))
NL2=list([None]*len(list_of_products))

# Getting the chunks and proceeding:

for chunk in reader:

    chunk.dropna(how='any', inplace=True)
#     chunk=chunk.drop('Unnamed: 0', axis=1)

    chunk['description_lower']=chunk['description'].str.lower()

    list_prod_list_ids=[None]*len(list_of_products)
    list_prod_list_descrip=[None]*len(list_of_products)
        
    for i, product in enumerate(list_of_products):

        list_prod_list_ids[i]=list(chunk[chunk['description_lower'].str.startswith(product)]['product_id'].unique())
        list_prod_list_descrip[i]=list(chunk[chunk['description_lower'].str.startswith(product)]['description_lower'].unique())
        
    for i, element in enumerate(list_prod_list_ids):
        
        if NL1[i]:    
            NL1.append(list_prod_list_ids[i])
            NL2.append(list_prod_list_descrip[i])
            
        else:
            NL1[i]=list_prod_list_ids[i]
            NL2[i]=list_prod_list_descrip[i]
    

In [13]:
rel_prod_list_ids=dict(zip(list_of_products,NL1))
rel_prod_list_descrip=dict(zip(list_of_products,NL2))

In [14]:
rel_prod_list_ids

{'croissant': [102.0,
  103.0,
  105.0,
  107.0,
  101.0,
  132.0,
  5001.0,
  100.0,
  214.0,
  189.0,
  198.0,
  197.0,
  9999.0,
  513.0,
  512.0,
  112.0],
 'croissant petit': [103.0, 102.0],
 'tarta mousse 3 chocolates': [9999.0, 453.0],
 'tarta de manzana 2º': [462.0, 9999.0],
 'palmera': [140.0, 182.0, 190.0, 9999.0, 141.0],
 'tarta opera': [9999.0, 414.0, 426.0, 427.0, 403.0, 14998.0, 402.0, 428.0],
 'postre fresas y mascarpone': [4511.0, 9999.0, 450.0],
 'milhojas frambuesa 2º': [459.0],
 'tortel': [112.0, 3352.0, 9999.0, 3375.0],
 'baguette': [115.0, 8739.0, 9999.0]}

##  3. FINAL LIST:

In [15]:
# At end, we reach the following conclussions:

dict_of_products={'croissant': 100, # serious doubts, if it is not the 100, then possibly it should be 100+101+102
                  'croissant petit': 103,
                  'tarta mousse 3 chocolates': 9999, # almost only for order, creating a new id for this product is suggested
                  'tarta de manzana 2º': 462,
                  'palmeras de trufa': 182, # palmeras: 140
                  'tarta opera': 414, # 9999, for order, mostly. If included, creating a new id for this product is suggested
                  'postre fresas y mascarpone':4511,
                  'milhojas frambuesa 2º': 459,
                  'torteles': 112,
                  'baguette':115}

In [16]:
dict_of_products

{'croissant': 100,
 'croissant petit': 103,
 'tarta mousse 3 chocolates': 9999,
 'tarta de manzana 2º': 462,
 'palmeras de trufa': 182,
 'tarta opera': 414,
 'postre fresas y mascarpone': 4511,
 'milhojas frambuesa 2º': 459,
 'torteles': 112,
 'baguette': 115}

## 4. CHANGING THE ID OF A PRODUCT:

We face now the problem that one of our products is using a code that describes the orders (as seen in previous scripts).

To avoid complexity in the code to come, we decide to change the id of our product to another one, taking care firstly that it is not currently in use.

In [17]:
df.loc[df['description_lower'].str.startswith('tarta mousse 3 chocolates'), 'product_id']=10002

In [18]:
df[df ['product_id']==10002]

,product_id,description,order_date,section,store,units_ordered,description_lower
888,10002.0,"Tarta mousse 3 chocolates del 3º, escrito en l...",2013-04-04,0,BmUP,"0,00","tarta mousse 3 chocolates del 3º, escrito en l..."
2340,10002.0,Tarta mousse 3 chocolates del 3º escrito encim...,2014-03-15,0,BmUP,"0,00",tarta mousse 3 chocolates del 3º escrito encim...
2531,10002.0,Tarta mousse 3 chocolates del 5º que ponga Fel...,2013-04-05,0,BmUP,"0,00",tarta mousse 3 chocolates del 5º que ponga fel...
2538,10002.0,TARTA MOUSSE 3 CHOCOLATES DEL SEGUNDO ESCRITO...,2013-04-05,0,BmUP,"0,00",tarta mousse 3 chocolates del segundo escrito...
38205,10002.0,TARTA MOUSSE 3 CHOCOLATES 4º,2012-12-18,0,BmUP,"0,00",tarta mousse 3 chocolates 4º
...,...,...,...,...,...,...,...
954996,10002.0,TARTA MOUSSE 3 CHOCOLATES DEL 4º,2008-05-23,0,BmUP,"0,00",tarta mousse 3 chocolates del 4º
958315,10002.0,"TARTA MOUSSE 3 CHOCOLATES DEL 2º ESCRITOS "" FE...",2008-05-25,0,BmUP,"0,00","tarta mousse 3 chocolates del 2º escritos "" fe..."
962928,10002.0,TARTA MOUSSE 3 CHOCOLATES 4º ESCRITO SOBRE LA...,2008-05-30,0,BmUP,"0,00",tarta mousse 3 chocolates 4º escrito sobre la...
963766,10002.0,TARTA MOUSSE 3 CHOCOLATES 4º,2008-05-31,0,BmUP,"0,00",tarta mousse 3 chocolates 4º


In [19]:
# We make this small arrangement also to the dict_of_products:

dict_of_products['tarta mousse 3 chocolates']= 10002 # New id created

In [20]:
dict_of_products

{'croissant': 100,
 'croissant petit': 103,
 'tarta mousse 3 chocolates': 10002,
 'tarta de manzana 2º': 462,
 'palmeras de trufa': 182,
 'tarta opera': 414,
 'postre fresas y mascarpone': 4511,
 'milhojas frambuesa 2º': 459,
 'torteles': 112,
 'baguette': 115}